In [2]:
import matplotlib.pyplot as plt

from markovBike.manager.manager import Manager
from markovBike.data_source.source import database_queries, get_stations_data
from markovBike.model.model import calculate_probability_matrix, BSiBSj, BSiBSj2, BSiTBj, TBiTBj

import pandas as pd
import numpy as np

In [ ]:
verbose = True

n_stations = 2_000

n_trips = 50_000_000

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

trips_raw = get_stations_data(database_queries(n_trips)['trips'], verbose=True)

# Call the calculate_probability_matrix function to get the probability matrix
probability_matrix = calculate_probability_matrix(trips_raw)

# Plot a heatmap of the probability matrix
fig, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(probability_matrix, cmap='coolwarm', ax=ax)

ax.set_title('Station Transition Probability Matrix')
ax.set_xlabel('End Station ID')
ax.set_ylabel('Start Station ID')

plt.show()

In [15]:
trips_raw = get_stations_data(database_queries(30_000_000)['trips'], verbose=True)

Bike station table with shape (30000000, 15). Columns are: 

tripduration                             Int64
starttime                  datetime64[ns, UTC]
stoptime                   datetime64[ns, UTC]
start_station_id                         Int64
start_station_name                      object
start_station_latitude                 float64
start_station_longitude                float64
end_station_id                           Int64
end_station_name                        object
end_station_latitude                   float64
end_station_longitude                  float64
bikeid                                   Int64
usertype                                object
birth_year                               Int64
gender                                  object
dtype: object



In [17]:
test = BSiBSj(trips_raw)

In [18]:
test.shape

(680, 680)

In [19]:
test

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])